In [1]:
from collections import namedtuple
import os
import tqdm
os.chdir("../")

In [2]:
from collections import namedtuple
from classifier_app.constants import *
from classifier_app.utils import read_yaml,create_directories
import os
import urllib.request as request
from zipfile import ZipFile

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_CLASSES: int
    params_WEIGHTS: str
    params_LEARNING_RATE: float
    params_INCLUDE_TOP: bool
    params_IMAGE_SIZE: list

In [4]:


class ConfigurationManager:


    def __init__(self, 
                    config_file_path = CONFIG_FILE_PATH,
                    params_filepath = PARAMS_FILE_PATH):
                    
                    self.config = read_yaml(config_file_path)
                    self.params = read_yaml(params_filepath)
                    
                    create_directories([self.config.artifacts_root])

    def PrepareBaseModelConfig(self)-> PrepareBaseModelConfig:
        config = self.config.prepare_Base_Model
        params = self.params

        create_directories([config.root_dir])

        prepareBaseModelConfig = PrepareBaseModelConfig(
                                root_dir=Path(config.root_dir),
                                base_model_path = Path(config.base_model_path),
                                updated_base_model_path= Path(config.updated_base_model_path),
                                params_CLASSES = params.CLASSES,
                                params_WEIGHTS = params.WEIGHTS,
                                params_LEARNING_RATE = params.LEARNING_RATE,
                                params_INCLUDE_TOP= params.INCLUDE_TOP,
                                params_IMAGE_SIZE=params.IMAGE_SIZE
                                )
        return prepareBaseModelConfig


    

In [8]:
import tensorflow as tf

class PrepareBaseModel:

    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config


    def get_base_model(self):

        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_IMAGE_SIZE,
            weights=self.config.params_WEIGHTS,
            include_top=self.config.params_INCLUDE_TOP,
            classes=self.config.params_CLASSES            
        )

        self.save_base_model(model_path= self.config.base_model_path, model= self.model)

    @staticmethod
    def _prepare_full_model(model ,classes,freeze_all,
                            freeze_till,learning_rate):
        
        if freeze_all:
            for layer in model.layers:
                model.trainable= False
        elif (freeze_till is not None) and (freeze_till>0):
            for layer in model.layers[:-freeze_till]:
                model.tainable = False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
                                            units= classes,
                                            activation= "softmax"
                                         )(flatten_in)

        full_model = tf.keras.Model(
            inputs= model.input,
            outputs = prediction
        )
        
        full_model.compile(
            optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ["accuracy"]
        )

        full_model.summary()

        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
                                    model = self.model,
                                    classes= self.config.params_CLASSES,
                                    freeze_all= True,
                                    freeze_till = None,
                                    learning_rate = self.config.params_LEARNING_RATE
                                     )

        self.save_base_model(model_path=self.config.updated_base_model_path,
                                model = self.full_model)

    @staticmethod
    def save_base_model(model_path: Path, model: tf.keras.Model):
        model.save(model_path)



In [9]:
try :

    config = ConfigurationManager()
    prepareBaseModel = PrepareBaseModel(config= config.PrepareBaseModelConfig())
    prepareBaseModel.get_base_model()
    prepareBaseModel.update_base_model()

except Exception as e:
    raise e

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   